In [2]:
import os                              
import numpy as np
from numpy import number                    
import pandas as pd                     
import matplotlib.pyplot as plt  
import math
from typing import List
from azure.quantum.optimization import Problem, ProblemType, Term
from azure.quantum.optimization import ParallelTempering
from azure.quantum.optimization import Term, SlcTerm

In [3]:
portfolio = pd.DataFrame(data['Time'])
for i in range(len(selected_stocks)):
       portfolio = portfolio.join(data[data.columns[1+index_prices_selected[i]]])
       
pr = portfolio[(portfolio['Time'] >= '2020-06-27')] \
    .drop(['Time'], axis=1) \
    .pct_change(fill_method='ffill')
pr.head()

pmu = pr.mean().to_numpy()
psigma = pr.cov().to_numpy()

NameError: name 'data' is not defined

In [ ]:
portfolio.plot(x ='Time', y = portfolio.columns[1:], kind = 'line')
plt.legend(bbox_to_anchor=(1,1))
plt.ylabel('Price in \u20B9')
plt.show()

In [ ]:
def find_expected_daily_return_sp(weights,portfolio_mu,n):
    '''
    Calculates the expected return for the optimised portfolio
    
    Parameters
    ----------
    weights: np.array
        weights of the selected stocks

    Returns
    -------
    expected_dialy_roi: float
        Expected daily return on inverstment of the optimised portfolio
    
    '''
    expected_daily_roi=0
    for i in range(0,n):
        t = weights[i] * portfolio_mu[i]
        expected_daily_roi+=t
    return expected_daily_roi

In [ ]:
from statistics import variance


def find_risk_sp(weights,portfolio_var,n):

    '''Calculates the risk for the optimised portfolio
    
    Parameters
    ----------
    B: float
        budget to be equally invested
    weights: np.array
        weights of the selected stocks

    Returns
    -------
    risk_perc: float
        percetnage risk value of the optimised portfolio with given budget
    '''
    
    var=0
    for i in range(0,n):
        for j in range(0,n):
            t = weights[i]  * weights[j] * portfolio_var[i][j]
            var+=t
    risk = var**0.5
    risk_perc = risk * 100
    return risk_perc

In [ ]:
def randomly_weighted_portfolios_with_selected_stocks(nof_portfolio, portfolio_return, portfolio_sigma):
    all_w_list = []
    all_b_list = []
    weight = np.random.random(len(selected_stocks))
    weight /= np.sum(weight)
    all_w_list.append(weight)
    random_portfolios = np.zeros((2,nof_portfolio))
    for i in range(nof_portfolio):
        weight = np.random.random(len(selected_stocks))
        weight /= np.sum(weight)
        all_w_list.append(weight)
        random_portfolios[0,i] = find_expected_daily_return_sp(weight, portfolio_return,len(selected_stocks))*30*100
        random_portfolios[1,i] = find_risk_sp(weight, portfolio_sigma,len(selected_stocks))
    return random_portfolios

In [ ]:
def visualize_efficient_frontier_for_selected_stocks():
    simulated_portfolios = randomly_weighted_portfolios_with_selected_stocks(25000,pmu,psigma)
    plt.figure(figsize=(10, 7))
    plt.scatter(simulated_portfolios[1,:],simulated_portfolios[0,:],cmap='YlGnBu', marker='o', s=10, alpha=0.3)
    plt.colorbar()
    plt.title('Simulated Portfolio Optimization based on Efficient Frontier')
    plt.ylabel('Expected Monthly retrun on investment')
    plt.xlabel('Risk Percentage')
    plt.show()

In [ ]:
from pypfopt import risk_models
from pypfopt import plotting
from pypfopt import expected_returns


all_prices=  data.drop(['Time'], axis=1)
sample_cov = risk_models.sample_cov(all_prices, frequency=252)
muc = expected_returns.mean_historical_return(all_prices)


w_classical = [0.0161,0.24485, 0.10312 , 0.23497, 0.01057, 0.20492, 0.13906, 0.04642]
w_classical_all = [0,0.0161,0,0.24485,0.10312,0,0.23497,0,0,0,0,0.01057,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.20492,0,0,0,0.13906,0,0,0,0,0,0,0,0,0,0,0.04642,0,0,0,0,0,0,0]
return_based_on_classical_w = find_expected_daily_return_sp(w_classical, muc, 8)*22

risk_based_on_classical_w  = find_risk_sp(w_classical_all, (sample_cov.reset_index(drop = True)).to_numpy(), 49)
print(return_based_on_classical_w,risk_based_on_classical_w)


In [ ]:
quantum_risk = find_risk()
quantum_return = edroi * 22 *100
print(quantum_risk,quantum_return)

In [ ]:
visualize_efficient_frontier_for_selected_stocks()